In [1]:
import matplotlib.pyplot as plt 
import matplotlib as mpl
from matplotlib.dates import (YEARLY, DateFormatter,
                              rrulewrapper, RRuleLocator, drange)
import numpy as np
import os
import datetime
import pandas as pd
import xarray as xr
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from PIL import Image

def read_sim(sim_path):
    
    f = xr.open_dataset(sim_path)
    # sim_kwt = f[q_vname1].values[:]
    sim_irf = f[q_vname2].values[:]
    sim_time = f[t_vname].dt.strftime(time_format).values[:]
    f.close()

    sim_start_str=sim_time[0]
    sim_end_str=sim_time[-1]
    return sim_irf, sim_start_str, sim_end_str

def read_obs(obs_path, sim_start_str, sim_end_str):
    obs = []
    obs_time = []
    with open(obs_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line and line.startswith('USGS'):
                splits = line.split()
                obs_time.append(splits[2])
                if len(splits)>3:#flow data exists
                    obs.append(float(splits[3])/35.315) #convert to m^3/s
                else:
                    obs.append(np.nan)

    # extract useful observed flow
    obs_start=(datetime.datetime.strptime(sim_start_str,time_format) - datetime.datetime.strptime(obs_time[0],time_format)).days
    obs_len=(datetime.datetime.strptime(sim_end_str,time_format) - datetime.datetime.strptime(sim_start_str,time_format)).days+1
    obs_useful = obs[obs_start:obs_start+obs_len]
    return obs_useful


def plot_hyd(obs, sim, sim_start_str, plot_start_date_summary, plot_end_date_summary,
                  title_str, obs_fmt, sim_fmt, output_dir, output_filename):
    
    fig_files = []
    for m in range(len(plot_start_date_summary)):

        fig_file = os.path.join(output_dir,'image_'+str(m+1)+'.png')
        fig_files.append(fig_file)

        row_num=1
        col_num=1        
        fig, ax = plt.subplots(row_num,col_num)
        fig.set_figwidth(6.5*col_num) #190mm
        fig.set_figheight(6.5*0.4*row_num) #5.61 heigh/width=3/4
        
        dpi_value=100
        formatter = DateFormatter('%m/%d/%y')
        # formatter = DateFormatter('%b-%y')

        plot_start_str=plot_start_date_summary[m]
        plot_end_str=plot_end_date_summary[m]

        plot_start=((datetime.datetime.strptime(plot_start_str,time_format)-datetime.datetime.strptime(sim_start_str,time_format)).days)
        plot_len=((datetime.datetime.strptime(plot_end_str,time_format)-datetime.datetime.strptime(plot_start_str,time_format)).days+1)

        plot_start_date=datetime.datetime.strptime(plot_start_str,time_format)
        t=[plot_start_date+datetime.timedelta(days=d) for d in range(0,plot_len)]
        t_plot=[mpl.dates.date2num(d) for d in t[0:plot_len]]        

        obs_plot=obs[plot_start+0:plot_start+plot_len]
    #         sim_kwt_plot=sim_kwt[plot_start+0:plot_start+plot_len]
        sim_irf_plot=sim[plot_start+0:plot_start+plot_len]

        ax.plot_date(t_plot, obs_plot, obs_fmt, linewidth=0.75, markersize=0.0, alpha=0.8, label='Observation')   
    #         ax.plot_date(t_plot, sim_kwt_plot, 'b--', linewidth=1.0, markersize=0.0, alpha=0.8, label='Prediction (KWT)') 
        ax.plot_date(t_plot, sim_irf_plot, sim_fmt, linewidth=0.75, markersize=0.0, alpha=0.8, label='Simulation (IRF)') 

        #axis, label, title, legend
        ax.set_title(title_str, fontsize='small', fontweight='bold')
        ax.set_xlabel('Date', fontsize='small')
        ax.set_ylabel('Flow $(m^3/s)$', fontsize='small')
        ax.xaxis.set_major_formatter(formatter)
        ax.xaxis.set_tick_params(labelsize='small')#rotation=30,
        ax.legend(loc='best', fontsize='x-small', framealpha=0.5) #loc='upper right', 

        plt.rc('xtick',labelsize='small')
        plt.rc('ytick',labelsize='small')                
        fig.tight_layout()
        fig.savefig(fig_file, dpi=dpi_value)
        plt.close(fig)      
        del fig

    # save as one figure for one calibration
    widths = []
    heights = []
    for fig_file in fig_files:
        im = Image.open(fig_file)
        widths.append(im.width)
        heights.append(im.height)

    max_width = max(widths)
    total_height = sum(heights)
    new_im = Image.new('RGB', (max_width, total_height))

    x_offset = 0
    for fig_file in fig_files:
        im = Image.open(fig_file)    
        new_im.paste(im, (0,x_offset))
        x_offset += im.size[1]
    new_im.save(os.path.join(output_dir,output_filename))
    [os.remove(os.path.join(output_dir,file)) for file in os.listdir(output_dir) if file.startswith('image')]
    
    return os.path.join(output_dir,output_filename)

#====================================================================================#
# Main script
# root_dir = os.getcwd()
root_dir = '/glade/u/home/hongli/work/sharp/basins/scripts_hongli/'
os.chdir(root_dir)
output_folder='step1_plot_calib_hydro'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
outputfile = 'hydrgraph.png'

sim_file='best/all.nc'
# q_vname1 = 'KWTroutedRunoff'
q_vname2 = 'IRFroutedRunoff'
t_vname = 'time'
time_format='%Y-%m-%d'
obs_file='model/Taylor_park_09107000.txt'

plot_start_date_summary=['2008-01-02']
plot_end_date_summary = ['2018-12-31']

calib_num = 6
alpha = 'a'
fig_file_summary = []

#====================================================================================#
# raw results (no calibration)
raw_outputfile = 'raw.png'
raw_dir = '../tayprk_huc12_metsim/'
print('raw')

sim_irf, sim_start_str, sim_end_str = read_sim(os.path.join(raw_dir, 'route/output/all.nc'))
obs_useful = read_obs(os.path.join(raw_dir, 'Taylor_park_09107000.txt'), sim_start_str, sim_end_str)

title_str = '('+chr(ord(alpha)) +') No calibration'
obs_fmt = 'k-'
sim_fmt = 'b--'
output_dir = os.path.join(root_dir, output_folder)

raw_plot = plot_scenario(obs = obs_useful, sim = sim_irf, sim_start_str = sim_start_str, 
                         plot_start_date_summary = plot_start_date_summary, 
                         plot_end_date_summary=plot_end_date_summary,
                         title_str = title_str, obs_fmt = obs_fmt, sim_fmt = sim_fmt, 
                         output_dir = output_dir, output_filename = raw_outputfile)
fig_file_summary.append(raw_plot)
del sim_irf, sim_start_str, sim_end_str, obs_useful

#====================================================================================#
# calibration results
for i in range(calib_num):
    calib_outputfile = 'calib_'+str(i+1)+'.png'
    calib_dir = '../tayprk_calib_'+str(i+1)+'/'
    print('calib '+str(i+1))
    
    sim_irf, sim_start_str, sim_end_str = read_sim(os.path.join(calib_dir, sim_file))
    obs_useful = read_obs(os.path.join(calib_dir, obs_file), sim_start_str, sim_end_str)

    title_str = '('+chr(ord(alpha) + i + 1) +') Calibration trial '+str(i+1)
    obs_fmt = 'k-'
    sim_fmt = 'r--'

    calib_plot = plot_hyd(obs = obs_useful, sim = sim_irf, sim_start_str = sim_start_str,
                          plot_start_date_summary = plot_start_date_summary,
                          plot_end_date_summary=plot_end_date_summary,
                          title_str = title_str, obs_fmt = obs_fmt, sim_fmt = sim_fmt, 
                          output_dir = output_dir, output_filename = calib_outputfile)
    fig_file_summary.append(calib_plot)
    del sim_irf, sim_start_str, sim_end_str, obs_useful

#====================================================================================#
# save as one figure for all
print('save')
widths = []
heights = []
for fig_file in fig_file_summary:
    im = Image.open(fig_file)
    widths.append(im.width)
    heights.append(im.height)

max_width = max(widths)
total_height = sum(heights)
new_im = Image.new('RGB', (max_width, total_height))

x_offset = 0
for fig_file in fig_file_summary:
    im = Image.open(fig_file)    
    new_im.paste(im, (0,x_offset))
    x_offset += im.size[1]
new_im.save(os.path.join(output_dir,outputfile))


raw
calib 1
calib 2
calib 3
calib 4
calib 5
calib 6
save
